1.  fit processed data with random forest and predict probabilities with test data
2.  load submission dataset, sort probabilities accordingly and save submission dataset

In [1]:
import re
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
data_path_raw = Path.cwd().parent / "data" / "raw"

data_path_preprocessed = Path.cwd().parent / "data" / "processed"


In [3]:
processed_df = pd.read_csv(data_path_preprocessed / "training_set_features__nominal_ordinal_WOE_Impute_Dropped_balanced.csv", index_col="respondent_id")
labels_df_balanced = pd.read_csv(data_path_preprocessed / "training_set_labels__balanced.csv", index_col="respondent_id")
test_df_processed = pd.read_csv(data_path_preprocessed / "test_set_features_nominal_ordinal_WOE_Impute_Dropped.csv", index_col="respondent_id")

In [4]:
## 1 fit processed data with random forest and predict probabilities with test data
# best practise for maximum number of features are a third of the total number of features 
# max depth = 10 because this performed best in individual decision tree (2_1 DecisionTreeClassification..)


max_feat = len(processed_df.columns)/3
max_feat

23.666666666666668

In [5]:
## 1

from sklearn.ensemble import RandomForestClassifier

import time 

start = time.time()
rand_forest = RandomForestClassifier(
    max_depth=10, 
    max_features = 24)
rand_forest.fit(processed_df, labels_df_balanced)
#test_predictions = rand_forest.predict(test_df_processed)
test_probability = rand_forest.predict_proba(test_df_processed)
end = time.time()
print(end - start)




15.72744607925415


In [6]:
## 1

y_preds = pd.DataFrame(
    {
        "h1n1_vaccine": test_probability[0][:, 1],
        "seasonal_vaccine": test_probability[1][:, 1],
    },
    index = test_df_processed.index
)
print("y_preds.shape:", y_preds.shape)
y_preds.head()

y_preds.shape: (26708, 2)


,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707,0.377616,0.273288
26708,0.102048,0.071731
26709,0.152960,0.742831
26710,0.751415,0.881080
26711,0.558055,0.502512


In [7]:
## 2 load submission dataset, sort probabilities accordingly and save submission dataset
submission_df = pd.read_csv(data_path_raw / "submission_format.csv", 
                            index_col="respondent_id")

In [8]:
## 2
np.testing.assert_array_equal(test_df_processed.index.values, 
                              submission_df.index.values)

In [9]:
## 2

submission_df["h1n1_vaccine"] = y_preds["h1n1_vaccine"]
submission_df["seasonal_vaccine"] = y_preds["seasonal_vaccine"]

In [10]:
## 2

output_path = Path.cwd().parent / "models" / "submissions"

submission_df.to_csv(output_path /'RandForest_balanced_data.csv', index=True)



In [11]:
# 0.8466 roc score on submission 